In [1]:
import numpy as np
from scipy.linalg import eigh_tridiagonal


In [2]:
def constr_tridiag(d,od):
    '''
    Constructs (ndim,ndim) symetric tridiagonal array given the diagonal and off-diagonal elements

    d : ndarray, shape (ndim,)
    The diagonal elements of the array.
    e : ndarray, shape (ndim-1,)
    The off-diagonal elements of the array.
    
    returns ndarray, shape (ndim,ndim)
    '''
    
    n = d.size
    tri = np.zeros((n,n))
    for idx1 in range(n-1):
        tri[idx1,idx1] = d[idx1]
        tri[idx1+1,idx1] = od[idx1]
        tri[idx1,idx1+1] = od[idx1]
    tri[-1,-1] = d[-1]
    return tri

In [8]:
def lanczos_eig(A,v0,m):
    '''
    Finds the lowest m eigen values and eigen vectors of a symmetric array

    A : ndarray, shape (ndim, ndim)
    Array to diagnolize.
    v0 : ndarray, shape (ndim,)
    A vector to start the lanczos iteration
    m : scalar
    the dim of the krylov subspace
    
    returns 
    E : ndarray, shape (m,) Eigenvalues
    W : ndarray, shape (ndim, m) Eigenvectors
    '''

    n = v0.size
    Q = np.zeros((m,n))
    
    v = np.zeros_like(v0)
    r = np.zeros_like(v0) # v1
    
    b = np.zeros((m,))
    a = np.zeros((m,))
    
    v0 = v0 / np.linalg.norm(v0)
    Q[0,:] = v0
    
    r = A @ v0
    a[0] = v0 @ r
    r = r - a[0]*v0
    b[0] = np.linalg.norm(r)
    
    error = 1e-16

    for i in range(1,m,1):
        v = Q[i-1,:]

        Q[i,:] = r / b[i-1]

        r = A @ Q[i,:] # |n>

        r = r - b[i-1]*v

        a[i] = (Q[i,:] @ r) # real?
        r = r - a[i]*Q[i,:]

        b[i] = np.linalg.norm(r)
        
        if b[i] < error:
            m = i
            print('smaller space found',m)
            break

    E,V = eigh_tridiagonal(a[:m],b[:m-1])
    Q = Q[:m]
    W = (Q.T @ V)
#     tri = constr_tridiag(a[:m],b[:m-1])
    return E, W


In [19]:
size = 8
perc0 = 0.2 # perccentage of non zero
mat = np.zeros(size*size)
mat[np.random.randint(0,size*size,size=int(size*size*perc0/1.5))] = 2*np.random.rand(int(size*size*perc0/1.5))-1
mat = mat.reshape(size,size)
mat = mat + mat.T # make symetric

x = np.zeros(size); x[0]=1
print(np.allclose(mat,mat.T),np.mean(mat != 0))
# mat

True 0.203125


In [20]:
E, W = lanczos_eig(mat,x,7)
W[:,0]

array([-0.67982572,  0.        ,  0.58644194,  0.        ,  0.09717863,
       -0.42951037,  0.        ,  0.        ])

In [22]:
np.linalg.eigh(mat)[1][:,0]

array([ 0.67982572,  0.        , -0.58644194,  0.        , -0.09717863,
        0.42951037,  0.        ,  0.        ])